In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.csv', parse_dates=['timestamp'])
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [3]:
df = df.select_dtypes(exclude=['object'])
#df['price_doc'] = np.log1p(df['price_doc'].values)

df.shape

(30471, 277)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.model_selection import GridSearchCV

In [5]:
area = df['price_doc']
df = df.drop(['timestamp','price_doc'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(df,
                                                    area,test_size=0.3,random_state=42)

In [6]:
x_train.columns

Index(['id', 'full_sq', 'life_sq', 'floor', 'max_floor', 'material',
       'build_year', 'num_room', 'kitch_sq', 'state',
       ...
       'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500',
       'cafe_count_5000_price_4000', 'cafe_count_5000_price_high',
       'big_church_count_5000', 'church_count_5000', 'mosque_count_5000',
       'leisure_count_5000', 'sport_count_5000', 'market_count_5000'],
      dtype='object', length=275)

In [7]:
dtrain = xgb.DMatrix(x_train, y_train, feature_names=x_train.columns)
dtest = xgb.DMatrix(x_test, y_test, feature_names=x_test.columns)

In [8]:
xgbr = xgb.XGBRegressor()
xgbr.fit(x_train,y_train)

np.sqrt(msle(y_test,xgbr.predict(x_test)))

0.47705940120644497

In [9]:
params = {'min_child_weight':[4,5],
         'silent': [1],
          'gamma':[0.3,0.5],
          'subsample':[0.6,0.8,1],
          'max_depth': [2,3,4]}

xgbr = xgb.XGBRegressor() 

grid = GridSearchCV(xgbr, params, n_jobs=-1)
grid.fit(x_train,y_train)

np.sqrt(msle(y_test,grid.best_estimator_.predict(x_test)))

0.47563472700123366

# Summary: ну учитывая топ1 лидерборда с 0.30087 по rmsle, результаты baseline (моего), ну и немного гридсеча выглядят не такими уж плачевными. Почитал другие решения, копипастить не стал, но контест интересный. Ребята фичи очень крутили. Встречал обучение на логарифмированном таргете (логично обучить сразу на логарифм это дело), это давало примерно 0.06-0.08 скора, что круто для таких малых телодвижений. Еще нужно учитывать, что ребята на паблик тесте получали свои скоры, а я по сути на val. На тесте просел бы, к гадалке не ходи.
